In [22]:
import abc
import numpy as np
from pyproteolizardvis.data import DataLoader
from pyproteolizardvis.point import ImsPointCloudVisualizer
from pyproteolizard.data import PyTimsDataHandle

dh = PyTimsDataHandle('../data/Mainz_1_pos_1_1_5087.d/')
slic = dh.get_slice_rt_range(rt_min=15.0, rt_max=20.0)

slic = slic.filter_ranged(mz_min=1090, mz_max=1100, scan_min=250, scan_max=1000)

class MyLoader(DataLoader):
    def __init__(self):
        super().__init__()
    
    def get_data(self):
        pass
    
    def on_load_clicked(self):
        pass
    
ml = MyLoader()
ml.display_widgets()

In [21]:
class MyCustomPointVis(ImsPointCloudVisualizer, abc.ABC):
    def __init__(self, data):
        super().__init__(data)
        
    def display_widgets(self):
        try:
            display(self.box)
        except Exception as e:
            print(e)
    
    def on_update_clicked(self, change):
        points_frame = self.data.get_precursor_points()
        points = points_frame[['frame', 'scan', 'mz', 'intensity']].values
        self.points_widget.data[0].x = points[:, 0]
        self.points_widget.data[0].y = points[:, 1]
        self.points_widget.data[0].z = points[:, 2]
        self.points_widget.data[0].marker = dict(size=self.point_size_slider.value,
                                                 color=np.log(points[:, 3]),
                                                 colorscale=self.color_scale.value,
                                                 line=dict(width=0),
                                                 opacity=self.opacity_slider.value)

        # TODO: calculate spacing MZ as a function of filter range
        self.points_widget.update_layout(margin=dict(l=0, r=0, b=0, t=0),
                                         scene={'xaxis': {'title': 'Rt-Index'},
                                                'yaxis': {'title': 'Mobility-Index'},
                                                'zaxis': {'title': 'm/z', 'dtick': 0.5}},
                                         template="plotly_white")

        
point_vis = MyCustomPointVis(slic)    

point_vis.display_widgets()